In [1]:
import transformers
import torch
import torch.nn.functional as F
import pandas as pd
import numpy as np
import os
import re
import random
import datasets
from datasets import load_dataset
from abc import ABC, abstractmethod
from typing import List, Optional, Tuple, Dict
from tqdm import tqdm
import pickle
from dotenv import load_dotenv
import openai
import json

from typing import List, Optional, Tuple, Dict
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import Dataset

import plotly.graph_objects as go
import plotly.express as px

from utils import untuple, eval_completions
from act_add.contrast_dataset import ContrastDataset
from scripts.get_activations import gen_pile_data, compare_token_lists, slice_acts
from act_add.model_wrapper import ModelWrapper
%load_ext autoreload
%autoreload 2

/home/ubuntu/anaconda3/envs/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = 'EleutherAI/pythia-12b'
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map = "auto")
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side = 'left')
tokenizer.pad_token = tokenizer.eos_token

# dataset_name = 'duped.12b'
# N_PROMPTS = 5000
# mem_data = load_dataset('EleutherAI/pythia-memorized-evals')[dataset_name]
# pile_prompts = gen_pile_data(N_PROMPTS, tokenizer, min_n_toks = 64)
mw = ModelWrapper(model, tokenizer)

Loading checkpoint shards: 100%|██████████| 3/3 [00:19<00:00,  6.39s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [25]:
mw = ModelWrapper(model, tokenizer)

In [26]:
# load probe
from probes import LRProbe
path = '../../gld/train-data-probes/data/12b/'

# open .pth file
probe_weights = torch.load(path + 'probe_weights_regged.pth')
probe = LRProbe.from_weights(probe_weights['net.0.weight'], probe_weights['net.0.bias'])
probe.to('cuda')

LRProbe(
  (net): Sequential(
    (0): Linear(in_features=5120, out_features=1, bias=True)
    (1): Sigmoid()
  )
)

In [27]:
from datasets import load_from_disk, Dataset, DatasetDict

dataset = load_from_disk(os.path.join(path, 'split_hf_token_dataset_vary_len_v2'))

# filter for labels == 1
prompts = [x[:32] for x in dataset['test'].filter(lambda x: x['labels'] == 1)['input_ids']]
prompts = tokenizer.batch_decode(prompts, skip_special_tokens=True)
prompts = list(set(prompts))

print(len(prompts))

1041


In [28]:
len(prompts)

1041

In [29]:
to_run = prompts[:10]
t_gen, t_tries, t_failures = mw.rej_sampl_generate(to_run, probe, 34, rej_sample_length=10, max_tries=25, log_rej_samples=True)

part 0 of generation


Genned (Banned) words [[',', 'pp', 'v', 'Object', ')', ' (', 'This', ')->', 'lp', 'V'], [' the', ' reader', ' and', ' ultimately', ' the', '\n', '//', ' form', 'atter', '.'], ['.', 'J', '.,', ' Griffith', ',', ' J', '.,', ' and', ' H', 'oyle'], [' "', 'sc', 'anned', '-', 'page', '"){', '.', '178', '}', '\n'], [')', '\n', '\n', 'The', ' website', ' of', ' the', ' V', 'ys', 'ok'], ['amsmath', '}', '\n', '                ', '\\', 'usepackage', '{', 'wasysym', '}', ' \n'], [' class', '="', 'Selection', 'Mark', '">', '&#', '160', ';</', 'span', '>'], ['47', '45', 'a', '\n', '\t', 'TIOC', 'M', 'GET', '                        ', '  '], ['www', '.', 'fl', 'c', '.', 'org', '/', '\n', '\n', 'The'], ['AP', 'Q', '_', 'GPIO', '_', 'P', 'INS', '(', '33', ');']]
Preds [1. 1. 1. 1. 0. 1. 1. 1. 0. 1.]
torch.Size([10, 42]) want shape batch x seq_len
New preds [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
New banned words [[',,\\', 'pp\n', 'v    ', 'Objectpp', ')v', ' (Object', 'This)', ')-> (', 'lpThis', 'V)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


New preds [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
New banned words [[',,\\),_,**', 'pp\n (i pp', 'v    This0if', 'Objectpp)->)ace', ')vlp ()\\', ' (ObjectVThis\n', 'This)tbl)->    ', ')-> (->lp(', 'lpThisQueryV (', 'V)->InterfacetblThis'], [' the a an its this', ' reader log input internal value', ' and entry value buffer.', ' ultimately.\n of This', ' the\nfunc a is', '\nfunc ( logger part', "// (entry's of", ' formentry *\n the', 'atter *Entry//\n', '.Entry) formatted//'], ['..J Jotten', 'J J.,, and', '.,. Griffith H C', ' Griffith and,oyle.', ', Griffith J andJ', ' J,., Griffith.', '., J and,\n', ' and. H JJ\xa0\n', ' H\noyle.\xa0\n', 'oyle\xa0\n,\n\xa0'], [' " “ (" "" \\"', 'scfig\\).b', 'anned:"){u\n\\', '-width00\n")', 'page="a![](\n', '"){909br\n', '.mm"j![](', '178"}+"cancerbr', '}\n\\004j', '\n<|endoftext|>u70cancer'], [')', '\n', '\n', 'The', ' website', ' of', ' the', ' V', 'ys', 'ok'], ['amsmathwasysymupgreekmathrsfsaligned', '}}\\}[}{math', '\nusepackagenr}', '           

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


...

New preds [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
New banned words [[',,\\),_,**,(,,*,{_, /*)\\\\I,,', 'pp\n (i ppvoidpa})pb=\r\nfID)', 'v    This0if **)){ (str G  ,,\\ \\', 'Objectpp)->)ace)& ((This_)UID(( pp\n\r\n', ')vlp ()\\ppThisvoid)-> \\_IV       ', ' (ObjectVThis\nv)->)(lp\n\tNULLUnknownoidpp(', 'This)tbl)->    ObjectlpriV\t*/*)\\v (', ')-> (->lp()Vidtbl((,)(\nObjectThis', 'lpThisQueryV ( ((tbl);->voidpp(    ))->', 'V)->InterfacetblThisvoid->(Query)(vchar(( (lp'], [" the a an its this ` info'entry Info Entry their your ourt", " reader log input internal valueentry andvalue'sLevel. native entry own the", " and entry value buffer.` additional' logger\nLog string's in Level", ' ultimately.\n of This and form in fieldfunc() type logger-And', " the\nfunc a is `atter '. ( and or tomemoryFormatter", '\nfunc ( logger partentry.format Ifinfo the default be form and', "// (entry's of.\n'. the * originaled usedatter logger", ' formentry *\n theDatafunc It loggerLogger message to in. valu

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


New preds [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
New banned words [[',,\\),_,**,(,,*,{_, /*)\\\\I,,,--,&', 'pp\n (i ppvoidpa})pb=\r\nfID)LI', 'v    This0if **)){ (str G  ,,\\ \\PVID', 'Objectpp)->)ace)& ((This_)UID(( pp\n\r\nOID_', ')vlp ()\\ppThisvoid)-> \\_IV        *IA', ' (ObjectVThis\nv)->)(lp\n\tNULLUnknownoidpp( ppctive', 'This)tbl)->    ObjectlpriV\t*/*)\\v (objScript', ')-> (->lp()Vidtbl((,)(\nObjectThis)--Error', 'lpThisQueryV ( ((tbl);->voidpp(    ))->LDebug', 'V)->InterfacetblThisvoid->(Query)(vchar(( (lpPV110'], [" the a an its this ` info'entry Info Entry their your ourt bytesa", " reader log input internal valueentry andvalue'sLevel. native entry own the of S", " and entry value buffer.` additional' logger\nLog string's in Level theentry", ' ultimately.\n of This and form in fieldfunc() type logger-And logLevel', " the\nfunc a is `atter '. ( and or tomemoryFormatter entry value", '\nfunc ( logger partentry.format Ifinfo the default be form and message,', "// (entry's o

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


New preds [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
New banned words [[',,\\),_,**,(,,*,{_, /*)\\\\I,,,--,&__,*,', 'pp\n (i ppvoidpa})pb=\r\nfID)LIoutpc', 'v    This0if **)){ (str G  ,,\\ \\PVID__)pp', 'Objectpp)->)ace)& ((This_)UID(( pp\n\r\nOID_ \\v', ')vlp ()\\ppThisvoid)-> \\_IV        *IA\nObject', ' (ObjectVThis\nv)->)(lp\n\tNULLUnknownoidpp( ppctive    )', 'This)tbl)->    ObjectlpriV\t*/*)\\v (objScript( ((', ')-> (->lp()Vidtbl((,)(\nObjectThis)--Error Thisvoid', 'lpThisQueryV ( ((tbl);->voidpp(    ))->LDebug ->)(', 'V)->InterfacetblThisvoid->(Query)(vchar(( (lpPV110 v((('], [" the a an its this ` info'entry Info Entry their your ourt bytesaentry google", " reader log input internal valueentry andvalue'sLevel. native entry own the of S\n logs", " and entry value buffer.` additional' logger\nLog string's in Level theentryfunc for", ' ultimately.\n of This and form in fieldfunc() type logger-And logLevel ( a', " the\nfunc a is `atter '. ( and or tomemoryFormatter entry valueentry g

In [ ]:
out = model.generate(tokenizer(prompts[0], return_tensors='pt')['input_ids'], max_length=1, do_sample=False, temperature=0.0, pad_token_id=tokenizer.eos_token_id, return_dict_in_generate=True, output_hidden_states=True, bad_words_ids=[[625], [253]])
out.sequences

tensor([[26851,   257,   900,   844,   304,   423,   310,   253, 17696, 11981,
         25113,   296,   323,   253,  5197,   457,    84,   443,  4539,   285,
         23325, 11981,  2285,    15,  3837, 23967,  3658,   187,   187,  1992,
          1239,   436]])

In [ ]:
len(out.sequences[0]), len(tokenizer(prompts[0], return_tensors='pt')['input_ids'][0])

(32, 31)